In [ ]:
#standard libraries
import csv
import os
import time

#custom libraries
import tensorflow as tf
import pandas as pd

In [ ]:
def read_file_format(filename_queue):
    reader = tf.TextLineReader(skip_header_lines=1)
    _, value = reader.read(filename_queue)

    record_defaults = [tf.constant([], dtype=tf.float32), tf.constant([], dtype=tf.int32)]
    _, col2 = tf.decode_csv(value, record_defaults=record_defaults)
#     col2 = tf.string_to_number(col2, out_type=tf.int32)
    
    example = tf.stack([col2])
    return example

In [ ]:
def input_pipeline(filenames, batch_size = 3, num_epochs = None, evaluation = False):   
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs, shuffle=False)

    example = read_file_format(filename_queue)
        
    return example

In [ ]:
def _activation_summary(x):
    tensor_name = x.name
    tensor_name = tensor_name.replace(':', '_')
    tensor_name = tensor_name.replace('(', '_')
    tensor_name = tensor_name.replace(')', '_')
    tensor_name = tensor_name.replace(' ', '_')

    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))

In [ ]:
#Need to factor so a block can replicate cells 
class LSTM_Block:
    
    def __init__(self, activation=tf.tanh, forget_bias=1.0, num_cells=1):
        self.block_num = block_num
        self.forget_bias = forget_bias
        self.activation = activation
        self.num_cells = num_cells
        self.state = tf.zeroes([batch_size, tf.shape(i_tensor)])
    
    def call(self, input_tensor):
        for c in range(num_cells):
            with tf.variable_scope('input{0}.{1}'.format(block_num, c)) as scope:
                i_tensor = tf.concat([input_tensor, self.cell_value])

                weights = tf.Variable(tf.random_normal(tf.shape(i_tensor)), name='weights', trainable=True)
                biases = tf.Variable(tf.random_normal(tf.shape(i_tensor)[1]), name='bias', trainable=True)

                lin_i = tf.matmul(i_tensor, weights) + biases
                _activation_summary(lin_i)

                gate_i = tf.sigmoid(lin_i)
                _activation_summary(gate_i)

            with tf.variable_scope('forget{0}.{1}'.format(block_num, c)) as scope:
                f_tensor = tf.concat([input_tensor, self.cell_value])

                weights = tf.Variable(tf.random_normal(tf.shape(f_tensor)), name='weights', trainable=True)
                biases = tf.Variable(tf.random_normal(tf.shape(f_tensor)[1]), name='bias', trainable=True)

                lin_f = tf.matmul(f_tensor, weights) + biases
                _activation_summary(lin_f)

                gate_f = tf.sigmoid(lin_f)
                _activation_summary(gate_f)

            with tf.variable_scope('cell{0}.{1}'.format(block_num, c)) as scope:
                weights = tf.Variable(tf.random_normal(tf.shape(input_tensor)), name='weights', trainable=True)
                biases = tf.Variable(tf.random_normal(tf.shape(input_tensor)[1]), name='bias', trainable=True)

                lin_state = tf.matmul(input_tensor, weights) + biases
                _activation_summary(lin_c)

                input_state = tf.matmul(gate_i, activation(lin_state))
                past_state = tf.matmul(gate_f, self.state)
                new_state = input_state + past_state
                _activation_summary(new_state)

                self.state = new_state

            with tf.variable_scope('output{0}.{1}'.format(block_num, c)) as scope:
                o_tensor = tf.concat([input_tensor, self.cell_value])

                weights = tf.Variable(tf.random_normal(tf.shape(o_tensor)), name='weights', trainable=True)
                biases = tf.Variable(tf.random_normal(tf.shape(o_tensor)[1]), name='bias', trainable=True)

                lin_o = tf.matmul(o_tensor, weights) + biases
                _activation_summary(lin_o)

                gate_o = tf.sigmoid(lin_o)
                _activation_summary(gate_o)

            with tf.variable_scope('hidden{0}.{1}'.format(block_num, c)) as scope:
                hidden = tf.matmul(gate_o, activation(self.cell_value))
                _activation_summary(hidden)

In [ ]:
def make_prediction(X):
    block1 = lstm_block()

In [ ]:
def calculate_loss(logits, labels):
    NLLCriterion = -tf.reduce_mean(tf.reduce_sum(tf.multiply(labels, tf.log(logits + 1e-10)), axis=1))

    tf.add_to_collection('losses', NLLCriterion)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [ ]:
pd.read_csv("{0}/{1}".format(data_path, train_file), nrows=10, header=None)

In [ ]:
global data_path
data_path = "data"

global vocab_file
vocab_file = "vocab1.csv"

global train_file
train_file = "train1.csv"

global batch_size
batch_size = 16

global seq_length
seq_length = 16

global num_epochs
num_epochs = 1

In [ ]:
with tf.Graph().as_default():
    x = tf.placeholder(tf.float32, shape=[None, 1])
    y_ = tf.placeholder(tf.float32, shape = [None, 1])

    y_hat = make_prediction(x)

    #need appropriate loss function
    loss = calculate_loss(y_hat, y_)

    #need to figure out gradient clipping in TF
    #need trunc. backprop through time
    train_op = train(loss, global_step=global_step)
    step = 0

    accuracy = evaluate_accuracy(y_hat, y_)

    example_feed = input_pipeline(["{0}/{1}".format(data_path, train_file)],
                                  batch_size = batch_size, num_epochs = num_epochs)
    
    with tf.Session() as sess:
        # Start populating the filename queue.
        
        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init_op)
        coord = tf.train.Coordinator()  
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)
        
        step = 0
        while not coord.should_stop():
#             try:
            start_time = time.time()                

            example_batch = sess.run(example_feed)
        
            duration = time.time() - start_time
            step += 1

            except (tf.errors.OutOfRangeError, tf.errors.InvalidArgumentError) as e:
           
                print('Done training for %d epochs, %d steps.' % (num_epochs, step))
                # When done, ask the threads to stop.
                coord.request_stop()

        
        # Wait for threads to finish.
        coord.join(threads)
        sess.close()

In [ ]:
global block_num
block_num = 0

In [ ]:
vocab = pd.read_csv("{0}/{1}".format(data_path, vocab_file),
                    header=None)
vocab_size = len(vocab)